In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cobra
from cobra.io import read_sbml_model



In [ ]:
model = read_sbml_model('Pan_putida2-Final.xml')
old_model = read_sbml_model('iJN746.xml')
model

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
COMPARTMENT_LABELS = {
    "c": "Cytoplasmic",
    "p": "Periplasmic",
    "e": "Extracellular",
}


COMPARTMENT_ALIASES = {
    "c0": "c",
    "p0": "p",
    "e0": "e",
}

In [8]:
COMPARTMENT_ALIASES.get('c', 'e')

'e'

In [18]:
def _norm_comp(comp_id: str) -> str:
    """Normalize compartment ID using aliases, return original if no alias."""
    return COMPARTMENT_ALIASES.get(comp_id, comp_id)

def is_transport_rxn(rxn) -> bool:
    """A reaction is 'transport' if it involves metabolites from ≥2 compartments."""
    comps = {_norm_comp(m.compartment) for m in rxn.metabolites}
    return len(comps) >= 2

def transport_bucket(rxn) -> str:
    """Place transport reactions into requested buckets based on participating compartments."""
    comps = {_norm_comp(m.compartment) for m in rxn.metabolites}
    if comps == {"c", "p"}:
        return "Cytoplasm to periplasm"
    if comps == {"p", "e"}:
        return "Periplasm to extracellular"
    if comps == {"c", "e"}:
        return "Cytoplasm to extracellular"
    # anything else multi-compartment (e.g., 3-way or non c/p/e) is not counted in requested buckets
    return "Other transport"

def gene_sets_by_role(m):
    """Return (all_genes, metabolic_genes, transport_genes) as sets of gene IDs.
    - metabolic_genes: genes associated with ≥1 non-transport reaction
    - transport_genes: genes associated with ≥1 transport reaction
    Note: a gene can appear in both sets.
    """
    all_genes = {g.id for g in m.genes}
    trans_genes, meta_genes = set(), set()
    for rxn in m.reactions:
        genes_here = {g.id for g in rxn.genes}
        if not genes_here:
            continue
        if is_transport_rxn(rxn):
            trans_genes.update(genes_here)
        else:
            meta_genes.update(genes_here)
    return all_genes, meta_genes, trans_genes

def metabolite_counts(m):
    """Counts for metabolites overall and by compartment."""
    mets = list(m.metabolites)
    total_unique = len(mets)
    by_comp = {label: 0 for label in COMPARTMENT_LABELS.values()}
    for met in mets:
        comp = _norm_comp(met.compartment)
        label = COMPARTMENT_LABELS.get(comp)
        if label:
            by_comp[label] += 1
    return total_unique, by_comp

def reaction_counts(m):
    """Counts for reactions overall and by compartment (dominant) & transport buckets.
    For 'Cytoplasmic/Periplasmic/Extracellular' reaction counts, a reaction is
    counted in a compartment if *all* participating metabolites are in that compartment.
    (Otherwise it is considered transport or multi-compartment.)
    """
    total = len(m.reactions)

    # Single-compartment reactions
    single_comp_counts = {label: 0 for label in COMPARTMENT_LABELS.values()}
    transport_total = 0
    transport_buckets = {
        "Cytoplasm to periplasm": 0,
        "Periplasm to extracellular": 0,
        "Cytoplasm to extracellular": 0,
    }

    for rxn in m.reactions:
        comps = {_norm_comp(met.compartment) for met in rxn.metabolites}
        # single-compartment?
        if len(comps) == 1:
            comp = next(iter(comps))
            label = COMPARTMENT_LABELS.get(comp)
            if label:
                single_comp_counts[label] += 1
            continue

        # multi-compartment -> transport (by our operational definition)
        if is_transport_rxn(rxn):
            transport_total += 1
            bucket = transport_bucket(rxn)
            if bucket in transport_buckets:
                transport_buckets[bucket] += 1

    return total, single_comp_counts, transport_total, transport_buckets

def unique_elements_counts(a_ids, b_ids):
    """Return counts unique to A, unique to B, and shared."""
    a_ids = [w.replace('_c','').replace('_e','').replace('_p','') for w in a_ids]
    b_ids = [w.replace('_c','').replace('_e','').replace('_p','') for w in b_ids]
    a_set, b_set = set(a_ids), set(b_ids)
    return len(a_set), len(b_set)

def build_summary(current, reference, current_name="model", reference_name="old_model"):
    # Genes
    cur_all, cur_meta, cur_trans = gene_sets_by_role(current)
    ref_all, ref_meta, ref_trans = gene_sets_by_role(reference)

    # Metabolites
    cur_met_total, cur_met_bycomp = metabolite_counts(current)
    ref_met_total, ref_met_bycomp = metabolite_counts(reference)

    cur_met_ids = [m.id for m in current.metabolites]
    ref_met_ids = [m.id for m in reference.metabolites]
    cur_unique_mets, ref_unique_mets = unique_elements_counts(cur_met_ids, ref_met_ids)

    # Reactions
    cur_rxn_total, cur_rxn_bycomp, cur_trans_total, cur_trans_buckets = reaction_counts(current)
    ref_rxn_total, ref_rxn_bycomp, ref_trans_total, ref_trans_buckets = reaction_counts(reference)

    # Table assembly (MultiIndex rows: Section / Subfeature)
    rows = []
    data = {current_name: [], reference_name: [], "Δ (" + current_name + " - " + reference_name + ")": []}

    def add_row(section, subfeature, cur_val, ref_val):
        rows.append((section, subfeature))
        data[current_name].append(cur_val)
        data[reference_name].append(ref_val)
        # Numeric delta if possible, else blank
        try:
            delta = (cur_val or 0) - (ref_val or 0)
        except TypeError:
            delta = ""
        data["Δ (" + current_name + " - " + reference_name + ")"].append(delta)

    # Genes
    add_row("Genes", "Total", len(cur_all), len(ref_all))
    add_row("Genes", "Metabolic", len(cur_meta), len(ref_meta))
    add_row("Genes", "Transport", len(cur_trans), len(ref_trans))

    # Metabolites
    add_row("Metabolites", "Total unique", cur_met_total, ref_met_total)
    add_row("Metabolites", "Unique Metabolites (vs other model)", cur_unique_mets, ref_unique_mets)
    for label in ["Cytoplasmic", "Periplasmic", "Extracellular"]:
        add_row("Metabolites", label, cur_met_bycomp.get(label, 0), ref_met_bycomp.get(label, 0))

    # Reactions (single-compartment counts + totals)
    add_row("Reactions", "Total", cur_rxn_total, ref_rxn_total)
    for label in ["Cytoplasmic", "Periplasmic", "Extracellular"]:
        add_row("Reactions", label, cur_rxn_bycomp.get(label, 0), ref_rxn_bycomp.get(label, 0))

    # Transport reactions
    add_row("Transport reactions", "Total", cur_trans_total, ref_trans_total)
    add_row("Transport reactions", "Cytoplasm to periplasm", cur_trans_buckets["Cytoplasm to periplasm"], ref_trans_buckets["Cytoplasm to periplasm"])
    add_row("Transport reactions", "Periplasm to extracellular", cur_trans_buckets["Periplasm to extracellular"], ref_trans_buckets["Periplasm to extracellular"])
    add_row("Transport reactions", "Cytoplasm to extracellular", cur_trans_buckets["Cytoplasm to extracellular"], ref_trans_buckets["Cytoplasm to extracellular"])

    idx = pd.MultiIndex.from_tuples(rows, names=["Feature", "Subfeature"])
    summary_df = pd.DataFrame(data, index=idx)

    # Optional: a small note about potential overlaps in gene categories
    note = (
        "Notes:\n"
        "- 'Transport reactions' are inferred as reactions involving ≥2 compartments based on metabolite compartments.\n"
        "- 'Metabolic' vs 'Transport' genes are assigned by the types of reactions they appear in; a gene can appear in both."
    )

    return summary_df, note

In [21]:
# ---- run the comparison ----------------------------------------------------
summary_df, notes = build_summary(model, old_model, current_name="This study", reference_name="iJN746")


In [22]:
summary_df

This study  iJN746  \
Feature             Subfeature                                                
Genes               Total                                      2326     746   
                    Metabolic                                  1810     575   
                    Transport                                   531     172   
Metabolites         Total unique                               2525     907   
                    Unique Metabolites (vs other model)        1723     708   
                    Cytoplasmic                                1527     695   
                    Periplasmic                                 496     124   
                    Extracellular                               407      88   
Reactions           Total                                      3301    1054   
                    Cytoplasmic                                1801     751   
                    Periplasmic                                 133      20   
                    Extracellular                               423      88   
Transport reactions Total                                       915     195   
                    Cytoplasm to periplasm                      486     106   
                    Periplasm to extracellular                  337      84   
                    Cytoplasm to extracellular                   40       5   

                                                         Δ (This study - iJN746)  
Feature             Subfeature                                                    
Genes               Total                                                   1580  
                    Metabolic                                               1235  
                    Transport                                                359  
Metabolites         Total unique                                            1618  
                    Unique Metabolites (vs other model)                     1015  
                    Cytoplasmic                                              832  
                    Periplasmic                                              372  
                    Extracellular                                            319  
Reactions           Total                                                   2247  
                    Cytoplasmic                                             1050  
                    Periplasmic                                              113  
                    Extracellular                                            335  
Transport reactions Total                                                    720  
                    Cytoplasm to periplasm                                   380  
                    Periplasm to extracellular                               253  
                    Cytoplasm to extracellular                                35